In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer,  make_column_selector as selector


from sklearn.metrics import plot_confusion_matrix, recall_score,\
    accuracy_score, precision_score, f1_score

from sklearn.dummy import DummyRegressor

from sklearn.metrics import plot_roc_curve

from xgboost import XGBRegressor
from sklearn.decomposition import PCA

from imblearn.pipeline import Pipeline as ImPipeline
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression,ElasticNet,Ridge,Lasso
from sklearn.svm import SVR

In [2]:
import math
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.losses import MeanSquaredLogarithmicError

In [3]:
df = pd.read_csv('./tb.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82 entries, 0 to 81
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  82 non-null     int64  
 1   Year        82 non-null     int64  
 2   G#          82 non-null     float64
 3   Week        82 non-null     float64
 4   Age         82 non-null     float64
 5   Tm          82 non-null     object 
 6   Home        82 non-null     float64
 7   Opp         82 non-null     object 
 8   Att         82 non-null     float64
 9   Cmp%        82 non-null     float64
 10  Sk          82 non-null     float64
 11  Ru_Att      82 non-null     float64
 12  RANK        82 non-null     int64  
 13  Dome        82 non-null     int64  
 14  Total       82 non-null     float64
dtypes: float64(9), int64(4), object(2)
memory usage: 9.7+ KB


In [14]:
df.head()

,Unnamed: 0,Year,G#,Week,Age,Tm,Home,Opp,Att,Cmp%,Sk,Ru_Att,RANK,Dome,Total
0,0,2017,1.0,1.0,40.035,NWE,1.0,KAN,36.0,44.44,3.0,2.0,15,0,10.68
1,1,2017,2.0,2.0,40.045,NWE,0.0,NOR,39.0,76.92,2.0,2.0,10,1,31.78
2,2,2017,3.0,3.0,40.052,NWE,1.0,HOU,35.0,71.43,5.0,1.0,32,1,36.72
3,3,2017,4.0,4.0,40.059,NWE,1.0,CAR,45.0,71.11,3.0,1.0,11,0,20.48
4,4,2017,5.0,5.0,40.063,NWE,0.0,TAM,40.0,75.00,3.0,2.0,23,0,14.62


In [5]:
df.corr()

,Unnamed: 0,Year,G#,Week,Age,Home,Att,Cmp%,Sk,Ru_Att,RANK,Dome,Total
Unnamed: 0,1.000000,0.980418,0.121336,0.121231,0.984045,-0.088645,0.216940,0.042831,-0.217035,0.052915,-0.024239,0.206437,0.139282
Year,0.980418,1.000000,-0.036799,-0.037102,0.999690,-0.081055,0.234742,0.042301,-0.209990,0.057510,-0.047394,0.237950,0.164270
G#,0.121336,-0.036799,1.000000,0.998610,-0.011948,0.058350,-0.099000,-0.035703,-0.103568,-0.140697,0.183856,-0.148770,-0.136311
Week,0.121231,-0.037102,0.998610,1.000000,-0.012224,0.053230,-0.108457,-0.036963,-0.107942,-0.136861,0.194004,-0.143176,-0.138737
Age,0.984045,0.999690,-0.011948,-0.012224,1.000000,-0.079797,0.232188,0.041509,-0.212795,0.054016,-0.042585,0.234766,0.161104
Home,-0.088645,-0.081055,0.058350,0.053230,-0.079797,1.000000,-0.105419,-0.046428,-0.049150,0.077351,-0.045018,0.069018,0.094474
Att,0.216940,0.234742,-0.099000,-0.108457,0.232188,-0.105419,1.000000,0.023946,0.106919,-0.013883,-0.210470,0.130607,0.274988
Cmp%,0.042831,0.042301,-0.035703,-0.036963,0.041509,-0.046428,0.023946,1.000000,-0.102608,0.139868,0.138632,0.092076,0.537310
Sk,-0.217035,-0.209990,-0.103568,-0.107942,-0.212795,-0.049150,0.106919,-0.102608,1.000000,0.050313,-0.120525,0.154908,0.005555
Ru_Att,0.052915,0.057510,-0.140697,-0.136861,0.054016,0.077351,-0.013883,0.139868,0.050313,1.000000,-0.110283,0.050767,0.127383


In [6]:
X = df.drop(['Total','Unnamed: 0'], axis=1)
y = df['Total']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42,test_size=.2)

In [7]:
#subpipes that scale numeric data and use one hot encoder on categorical 
subpipe_num = Pipeline(steps=[
    ('num_impute', SimpleImputer(strategy='median')),
    ('ss', StandardScaler())
])


subpipe_cat = Pipeline(steps=[
    ('cat_impute',SimpleImputer(strategy='most_frequent')),
    ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))
])

In [16]:
cat_feat = ['Tm', 'Opp']

In [17]:
#Create a column transformer to apply the subpipes and transform the data
CT = ColumnTransformer(transformers=[
    ('subpipe_num', subpipe_num, selector(dtype_include=np.number)),
     ('subpipe_cat', subpipe_cat, cat_feat)], remainder='passthrough')

In [8]:
# #Create a column transformer to apply the subpipes and transform the data
# CT = ColumnTransformer(transformers=[
#     ('subpipe_num', subpipe_num, selector(dtype_include=np.number)),
#      ('subpipe_cat', subpipe_cat, selector(dtype_include=object))], remainder='passthrough')

In [18]:
#Create a pipeline for dummy model
dummy_model_pipe = Pipeline(steps=[
    ('ct', CT),
    ('dum', DummyRegressor(strategy='median'))
])

In [19]:
dummy_model_pipe.fit(X_train, y_train)
dummy_model_pipe.score(X_train,y_train)

-0.007672389179546046

# Simple Regression Models

In [11]:
regressors = [
    
    LinearRegression(),
    Ridge(),
    Lasso(),
    SVR(),
]

In [12]:
for regressor in regressors:
    steps = [
        ('ct', CT),
        ('rg', regressor)
    ]
    pipeline = Pipeline(steps)
    pipeline.fit(X_train, y_train)   
    print(regressor)
    print("model score: %.3f" % pipeline.score(X_train, y_train))
    

LinearRegression()
model score: 0.800
Ridge()
model score: 0.775
Lasso()
model score: 0.563
SVR()
model score: 0.311


In [24]:
import xgboost as xgb

In [26]:
GB = xgb.XGBRegressor(random_state=42)

In [27]:
xgb_model_pipe = Pipeline(steps=[
    ('ct', CT),
    ('xgb', GB)
])

In [28]:
xgb_model_pipe.fit(X_train, y_train)
xgb_model_pipe.score(X_train,y_train)

0.9999999858342301

In [ ]:
transformer_pipe = Pipeline(steps=[
    ('ct', CT)
])

# Neural Network 

In [36]:
tm_data = df.copy()

In [37]:
tm_data = pd.get_dummies(tm_data['Tm'])

In [38]:
opp_data = df.copy()

In [39]:
opp_data = pd.get_dummies(opp_data['Opp'])

In [41]:
opp_data

,ATL,BAL,BUF,CAR,CHI,CIN,CLE,DAL,DEN,DET,...,MIN,NOR,NWE,NYG,NYJ,PHI,PIT,TAM,TEN,WAS
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
78,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
79,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
80,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
encoded_df = pd.concat([df, tm_data, opp_data], axis=1)

In [44]:
X = encoded_df.drop(['Total','Unnamed: 0', 'Tm', 'Opp'], axis=1)
y = encoded_df['Total']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42,test_size=.2)

In [45]:
X_train

,Year,G#,Week,Age,Home,Att,Cmp%,Sk,Ru_Att,RANK,...,MIN,NOR,NWE,NYG,NYJ,PHI,PIT,TAM,TEN,WAS
62,2020,15.0,16.0,43.145,0.0,27.0,81.48,1.0,0.0,32,...,0,0,0,0,0,0,0,0,0,0
56,2020,8.0,8.0,43.091,0.0,40.0,70.00,2.0,1.0,8,...,0,0,0,1,0,0,0,0,0,0
40,2019,8.0,8.0,42.085,1.0,36.0,55.56,3.0,0.0,20,...,0,0,0,0,0,0,0,0,0,0
9,2017,9.0,10.0,40.101,0.0,34.0,73.53,1.0,1.0,23,...,0,0,0,0,0,0,0,0,0,0
78,2021,17.0,18.0,44.159,1.0,37.0,78.38,1.0,0.0,21,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20,2018,5.0,5.0,41.062,1.0,44.0,77.27,0.0,3.0,10,...,0,0,0,0,0,0,0,0,0,0
60,2020,14.0,15.0,43.139,0.0,45.0,68.89,3.0,2.0,19,...,0,0,0,0,0,0,0,0,0,0
71,2021,8.0,8.0,44.089,0.0,40.0,70.00,3.0,1.0,4,...,0,1,0,0,0,0,0,0,0,0
14,2017,14.0,15.0,40.136,0.0,35.0,62.86,2.0,2.0,7,...,0,0,0,0,0,0,1,0,0,0


In [47]:
def scale_datasets(X_train, X_test):
    standard_scaler = StandardScaler()
    X_train_scaled = pd.DataFrame(
        standard_scaler.fit_transform(X_train),
        columns=X_train.columns
  )
    X_test_scaled = pd.DataFrame(
        standard_scaler.transform(X_test),
        columns = X_test.columns
  )
    return X_train_scaled, X_test_scaled
X_train_scaled, X_test_scaled = scale_datasets(X_train, X_test)

In [49]:
hidden_units1 = 32
hidden_units2 = 64
hidden_units3 = 128
learning_rate = 0.01
# Creating model using the Sequential in tensorflow
def build_model_using_sequential():
    model = Sequential([
        Dense(hidden_units1, kernel_initializer='normal', activation='relu'),
        Dropout(0.2),
        Dense(hidden_units2, kernel_initializer='normal', activation='relu'),
        Dropout(0.2),
        Dense(hidden_units3, kernel_initializer='normal', activation='relu'),
        Dense(1, kernel_initializer='normal', activation='linear')
      ])
    return model
# build the model
model = build_model_using_sequential()

In [51]:
# loss function
msle = MeanSquaredLogarithmicError()
model.compile(
    loss=msle, 
    optimizer=Adam(learning_rate=learning_rate), 
    metrics=['accuracy']
)
# train the model
history = model.fit(
    X_train_scaled.values, 
    y_train.values, 
    epochs=10, 
    batch_size=64,
    validation_split=0.2
)

Epoch 1/10
1/1 [==============================] - 0s 132ms/step - loss: 8.2396 - accuracy: 0.0000e+00 - val_loss: 2.6145 - val_accuracy: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 0s 18ms/step - loss: 1.5412 - accuracy: 0.0000e+00 - val_loss: 0.2060 - val_accuracy: 0.0000e+00
Epoch 3/10
1/1 [==============================] - 0s 17ms/step - loss: 0.2656 - accuracy: 0.0000e+00 - val_loss: 0.2933 - val_accuracy: 0.0000e+00
Epoch 4/10
1/1 [==============================] - 0s 17ms/step - loss: 0.9445 - accuracy: 0.0000e+00 - val_loss: 0.6127 - val_accuracy: 0.0000e+00
Epoch 5/10
1/1 [==============================] - 0s 18ms/step - loss: 1.4294 - accuracy: 0.0000e+00 - val_loss: 0.7018 - val_accuracy: 0.0000e+00
Epoch 6/10
1/1 [==============================] - 0s 19ms/step - loss: 1.3512 - accuracy: 0.0000e+00 - val_loss: 0.6101 - val_accuracy: 0.0000e+00
Epoch 7/10
1/1 [==============================] - 0s 19ms/step - loss: 0.9787 - accuracy: 0.0000e+00 - val_loss: 0.46